# **Importação das Bibliotecas Necessárias**

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# **Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Definir de Parametros e Imagens**

In [ ]:
# Define paths
images_dir = '/content/drive/MyDrive/Modelos/dataset/images'
masks_dir = '/content/drive/MyDrive/Modelos/dataset/masks'

In [ ]:
# Parameters
img_height, img_width = 512, 512  # Desired dimensions
batch_size = 8  # Adjust based on memory constraints

# **Carregamento e Pré-processamento de imagens**

In [ ]:
# Function to load and preprocess images and masks
def load_and_preprocess_image(img_path, mask_path, target_size=(img_height, img_width)):
    image = load_img(img_path, target_size=target_size)
    image = img_to_array(image) / 255.0  # Normalize to [0, 1]

    mask = load_img(mask_path, target_size=target_size, color_mode="grayscale")
    mask = img_to_array(mask) / 255.0  # Normalize to [0, 1]
    mask = np.round(mask)  # Ensure mask is binary (0 or 1)

    return image, mask

In [ ]:
# Get list of image and mask files
image_files = sorted(os.listdir(images_dir))
mask_files = sorted(os.listdir(masks_dir))

In [ ]:
# Ensure the number of images and masks match
assert len(image_files) == len(mask_files)

In [ ]:
# Split the dataset into training and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(
    image_files, mask_files, test_size=0.2, random_state=42
)

In [ ]:
# Load and preprocess all data
train_images = [load_and_preprocess_image(os.path.join(images_dir, img), os.path.join(masks_dir, msk)) for img, msk in zip(train_images, train_masks)]
val_images = [load_and_preprocess_image(os.path.join(images_dir, img), os.path.join(masks_dir, msk)) for img, msk in zip(val_images, val_masks)]

In [ ]:
# Separate images and masks
train_images, train_masks = zip(*train_images)
val_images, val_masks = zip(*val_images)

In [ ]:
# Convert lists to numpy arrays
train_images = np.array(train_images)
train_masks = np.array(train_masks)
val_images = np.array(val_images)
val_masks = np.array(val_masks)

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_masks)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_masks)).batch(batch_size)

# **Construção do Modelo**

In [ ]:
# Define U-Net model
def build_unet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

 # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same')(c9)

    model = models.Model(inputs, outputs)
    return model


# **Treino**

In [ ]:
# Build U-Net model
unet_model = build_unet((img_height, img_width, 3))

unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

unet_history = unet_model.fit(train_dataset, epochs=30, validation_data=val_dataset)

# **Visualização**

In [ ]:
# Function to plot training history
def plot_history(history, title):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title(f'Training and Validation Accuracy: {title}')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title(f'Training and Validation Loss: {title}')
    plt.legend()

    plt.show()


In [ ]:
plot_history(unet_history, 'U-Net')

# **Teste e Previão**

In [ ]:
def calculate_iou(pred_mask, true_mask):
    """
    Calculate the Intersection over Union (IoU) score between the predicted mask and the ground truth mask.

    Parameters:
    - pred_mask: numpy array of the predicted mask.
    - true_mask: numpy array of the ground truth mask.

    Returns:
    - iou: IoU score.
    """
    # Ensure masks are binary
    pred_mask = np.round(pred_mask).astype(np.uint8)
    true_mask = np.round(true_mask).astype(np.uint8)

    # Calculate Intersection and Union
    intersection = np.logical_and(pred_mask, true_mask).sum()
    union = np.logical_or(pred_mask, true_mask).sum()

    # Calculate IoU
    if union == 0:
        return 0.0  # Avoid division by zero
    iou = intersection / union
    return iou

In [ ]:
def predict_and_display_mask(model, image_path, ground_truth_path, target_size=(img_height, img_width)):
    """
    Predict and display the mask for an external image using the trained model and compare it with the ground truth mask.
    Display the IoU score as well.

    Parameters:
    - model: Trained U-Net model.
    - image_path: Path to the external image.
    - ground_truth_path: Path to the ground truth mask.
    - target_size: Tuple of target image size (height, width).
    """
    # Load and preprocess the image
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image) / 255.0  # Normalize to [0, 1]
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

    # Predict the mask
    predicted_mask = model.predict(image_array)[0]

    # Post-process the mask (if necessary, e.g., thresholding)
    predicted_mask = np.round(predicted_mask).astype(np.uint8).squeeze()  # Remove batch dimension

    # Load and preprocess the ground truth mask
    ground_truth_mask = load_img(ground_truth_path, target_size=target_size, color_mode="grayscale")
    ground_truth_mask = img_to_array(ground_truth_mask) / 255.0  # Normalize to [0, 1]
    ground_truth_mask = np.round(ground_truth_mask).astype(np.uint8).squeeze()  # Ensure mask is binary (0 or 1) and remove single channel dimension

    # Calculate IoU score
    iou_score = calculate_iou(predicted_mask, ground_truth_mask)
    print(f"IoU Score: {iou_score:.4f}")

    # Display the image, the ground truth mask, and the predicted mask
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("Original Image")

    plt.subplot(1, 3, 2)
    plt.imshow(predicted_mask, cmap='gray')
    plt.title("Predicted Mask")

    plt.subplot(1, 3, 3)
    plt.imshow(ground_truth_mask, cmap='gray')
    plt.title("Ground Truth Mask")


    plt.show()

In [ ]:
# Predict and display mask for an external image
image_path = '/content/drive/MyDrive/Modelos/dataset/test_images/daea4860-a296-4f6d-acef-996ab0882676.png'
ground_truth_path = '/content/drive/MyDrive/Modelos/dataset/test_masks/daea4860-a296-4f6d-acef-996ab0882676.png'
predict_and_display_mask(unet_model, image_path, ground_truth_path)

# **Guardar o Modelo Treinado**

In [ ]:
# Function to save the model
def save_model(model, save_path):
    """
    Save the trained model to the specified path.

    Parameters:
    - model: Trained Keras model.
    - save_path: Path to save the model.
    """
    model.save(save_path)
    print(f"Model saved to {save_path}")
